In [10]:
pip install keras-tuner

     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=af448d9c56a357069e900173a9d2b6d7924ad5665fd8c578913d3007331de1ca
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=e4189b8be1055a63a30210bef564ea7a9cea1f8a4690463c1b415e84c3880a85
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D,Flatten,Dense,Dropout,MaxPooling2D
from keras import backend as k


Using TensorFlow backend.


In [ ]:
print(tf.__version__)

2.2.0


In [3]:
img_rows=32
img_cols=32

In [4]:
### Split the data as train and test
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [5]:
if k.image_data_format() =='channel_first':
    x_train = x_train.reshape(x_train.shape[0],3,img_rows,img_cols)
    x_test = x_test.reshape(x_test.shape[0],3,img_rows,img_cols)
    input_shape=(3,img_rows,img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0],img_rows,img_cols,3)
    x_test = x_test.reshape(x_test.shape[0],img_rows,img_cols,3)
    input_shape=(img_rows,img_cols,3)

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train/=255
x_test/=255
print("reshape image",x_train.shape[0])

reshape image 50000


In [7]:
### Convert the class vectors to binary classes
## utils. to_categorical is used to convert array of labeled data(from 0 to nb_classes-1) to one-hot vector.
y_train = keras.utils.to_categorical(y_train,10)
y_test = keras.utils.to_categorical(y_test,10)

In [13]:
def build_model(hyper_para):
  model=keras.Sequential([
    keras.layers.Conv2D(filters=hyper_para.Int('conv_1_filter',min_value=32,max_value=128,step=10),
        kernel_size=hyper_para.Choice('conv_1_kernel',values=[3,5]),
        activation='relu',
        input_shape=(32,32,1)
    ),
    keras.layers.Conv2D(filters=hyper_para.Int('conv2d_2_filter',min_value=32,max_value=64,step=15),
      kernel_size=hyper_para.Choice('conv_2_kernel',values=[3,5]),
      activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
      units=hyper_para.Int('dense_1_units',min_value=32,max_value=64,step=16),
      activation='relu'
    ),
    keras.layers.Dense(10,activation='softmax')
  ])
  model.compile(optimizer=keras.optimizers.Adam(hyper_para.Choice('learning_rate',values=[1e-2,1e-3])),
               loss='sparse_categorical_crossentropy',metrics=['accuracy'])

  return model

In [14]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [15]:
tuner_model = RandomSearch(build_model,objective=['val_accuracy'],max_trials=5)

RuntimeError: ignored